Imports, loading DFs

In [63]:
import pandas as pd
import re
import os
#Their AI
ai_df = pd.read_csv('Data/remoteVisibAI.csv')
ai_df[['Date','Time']] = ai_df['Time [UTC]'].str.split(' ', expand=True)
ai_df['Date'] = pd.to_datetime(ai_df['Date'])
ai_df = ai_df.drop(['min visibility unit', 'visibility unit', 'Time [UTC]'], axis=1)
ai_df = ai_df[['Date','Time', 'visibility','min visibility', 'direction visibility']]
ai_df = ai_df.rename(columns={"visibility": "VisAI", "min visibility": "MinVis", "direction visibility": "MinVisDirOs"})
c1 = (ai_df['Date'] >= '2019-11-01') & (ai_df['Date'] <= '2020-02-29')
c2 = (ai_df['Time'] >= '05:20:00') & (ai_df['Time'] <= '17:00:00')
ai_df['VisAI'] = ai_df['VisAI'].astype(int)
ai_df = ai_df[c1 & c2]
#Observer
obs_df = pd.read_csv('Data/remoteVisibObs.csv',encoding='latin',sep=';')
obs_df = obs_df.dropna(subset=['VisObs'])
obs_df[['Date','Time']] = obs_df['Timestamp'].str.split('_', expand=True)
obs_df = obs_df.drop(['Timestamp', 'Note', 'DONE.Who'], axis=1)
obs_df = obs_df[['Date','Time', 'VisObs', 'MinVisObs', 'MinVisDirObs']]
def rearrange_date(date_str,dayFirst):
    match = re.match(r'(\d{1,2})[./](\d{1,2})[./](\d{4})', date_str)
    if match and dayFirst:
        return f"{match.group(3)}-{match.group(2).zfill(2)}-{match.group(1).zfill(2)}"
    elif match and not dayFirst:
        return f"{match.group(3)}-{match.group(1).zfill(2)}-{match.group(2).zfill(2)}"
    else:
        return date_str
obs_df['Date'] = obs_df['Date'].apply(lambda x: rearrange_date(x,dayFirst=False)) 
c3 = (obs_df['Date'] >= '2019-11-01') & (obs_df['Date'] <= '2020-02-29')
c4 = (obs_df['Time'] >= '05:20:00') & (obs_df['Time'] <= '17:00:00')
obs_df = obs_df[c3 & c4]
#Sensors
sen_df = pd.read_csv("Data/nov_feb_2019_SENZOR.csv")
sen_df[['Date','Time']] = sen_df['Time [UTC]'].str.split(' ', expand=True)
sen_df['Date'] = sen_df['Date'].apply(lambda x: rearrange_date(x,dayFirst=True)) 
sen_df = sen_df.drop(['Time [UTC]', 'VIS 2Min Avg [m]'], axis=1)
sen_df = sen_df.rename(columns={"VIS 1Min Avg [m]": "VisSen"})
sen_df = sen_df[['Date','Time', 'VisSen']]
c5 = (sen_df['Time'] >= '05:20:00') & (sen_df['Time'] <= '17:00:00')
sen_df['Time'] = sen_df['Time'].str.slice(stop=-3)
sen_df = sen_df[c5].groupby(['Date','Time'])['VisSen'].mean().reset_index()
sen_df['VisSen'] = sen_df['VisSen'].astype(int)
#Classes
classes_df = pd.read_csv('classes.csv')
class_distance_df = pd.read_csv('class_distance.csv')
merged_df = pd.merge(classes_df, class_distance_df, left_on=['Name'], right_on=['Class_name'], how='inner')
merged_df = merged_df.drop(columns=['Name', 'Class_name'])
merged_df.to_csv('class_info.csv', index=False)
info_df = pd.read_csv('class_info.csv')
info_df['distance'] = info_df['distance'].astype(str).str[:-2].astype('int64')
info_df = info_df.sort_values(by=['direction','distance'])
info_df['direction'] = info_df['direction'].astype('str').str.zfill(3)

/tmp/ipykernel_7720/1195617394.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ai_df['Date'] = pd.to_datetime(ai_df['Date'])


Importing predicted labels

In [60]:
#og_path = 'scripts/dp_scripts/labels/'
og_path = 'runs/detect/dp/exp/labels/'
i = 0
files = os.listdir(og_path)
pred_df = pd.DataFrame(columns=['Date','Time','Direction','VisYOLO'])
for file in files:
    path = og_path + file
    temp_df = pd.read_csv(path, names=['ID','x_center', 'y_center', 'width', 'height', 'conf'], sep='\s+')
    temp_df = temp_df[['ID','conf']]
    direction = file[24:27]
    date_str = file[28:36]
    time_str = file[36:40]
    date_str = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    time_str = f"{time_str[0:2]}:{time_str[2:]}"
    direction_set =  info_df['ID'][(info_df['direction'] == direction)]
    cond = temp_df['ID'].isin(direction_set)
    temp_df = temp_df[cond]
    final_df = pd.merge(left=info_df, right=temp_df, on='ID')
    final_df = final_df[final_df['conf'] >= 0.45]
    max_dist = final_df['distance'].max()
    pred_df.loc[i] = [date_str,time_str,direction,max_dist]
    i+=1

,Date,Time,Direction,MaxDist
0,2020-01-27,15:35,180,5900.0
1,2020-02-23,06:10,135,4500.0
2,2019-11-07,15:40,135,4500.0
3,2019-11-28,13:25,315,2400.0
4,2020-02-03,11:20,135,4500.0


In [64]:
pred_df = pd.read_csv('pred_df.csv')
pred_df.drop(columns='ID',axis=1, inplace=True)
pred_df.sort_values(by=['Date', 'Time', 'MaxDist'], ascending=[True, True, False], inplace=True)
result_df = pred_df.groupby(['Date', 'Time']).nth(3).reset_index(drop=True)
result_df.drop(columns='Direction',axis=1,inplace=True)

In [65]:
comp_df = pd.merge(obs_df, result_df, on=['Date','Time'])
comp_df = pd.merge(comp_df, sen_df,on=['Date','Time'])
comp_df['VisObs'] = comp_df['VisObs'].astype(int)
comp_df['MaxDist']
#TODO uprav datatypes
floor_dist = 10000
comp_df.loc[comp_df['VisObs'] > floor_dist, 'VisObs'] = floor_dist
comp_df.loc[comp_df['MaxDist'] > floor_dist, 'MaxDist'] = floor_dist
comp_df.loc[comp_df['VisSen'] > floor_dist, 'VisSen'] = floor_dist
#comp_df[['VisObs','MaxDist']][comp_df['VisObs'] < 1000]
comp_df['Difference'] = (comp_df['VisSen'] - comp_df['MaxDist']).abs()
condition1 = (comp_df['MaxDist'] < 600) & (comp_df['Difference'] > 50)
condition2 = (comp_df['MaxDist'].between(600, 1500)) & (comp_df['Difference'] > comp_df['VisSen'] * 0.1)
condition3 = (comp_df['MaxDist'].between(1500, 10000)) & (comp_df['Difference'] > comp_df['VisSen'] * 0.2)

comp_df[(condition1|condition2|condition3)]


,Date,Time,VisObs,MinVisObs,MinVisDirObs,MaxDist,VisSen,Difference
250,2019-11-02,15:55,7600,1900,S,7900.0,10000,2100.0
255,2019-11-03,05:50,7600,NaN,NaN,5900.0,10000,4100.0
256,2019-11-03,05:55,7600,NaN,NaN,7900.0,10000,2100.0
257,2019-11-03,06:00,5900,NaN,NaN,6000.0,10000,4000.0
258,2019-11-03,06:05,6800,NaN,NaN,5900.0,10000,4100.0
...,...,...,...,...,...,...,...,...
3347,2020-02-28,15:15,2400,675,E,4500.0,10000,5500.0
3348,2020-02-28,15:20,2900,1300,SE,5900.0,10000,4100.0
3349,2020-02-28,15:25,4200,1500,N,7800.0,10000,2200.0
3350,2020-02-28,15:30,4200,1500,N,7900.0,10000,2100.0


In [66]:
metar_df = pd.read_csv('Data/metar_speci_LZTT_2018_2021_decoded.csv')
metar_df = metar_df[['Date(UTC)','Poprad-Tatry Visibility [m]']]
metar_df = metar_df.drop(index=0, axis=0)
metar_df[['Date','Time']] = metar_df['Date(UTC)'].str.split(' ', expand=True)
metar_df.rename(columns={'Poprad-Tatry Visibility [m]':'VisMetar'},inplace=True)
metar_df = metar_df[['Date','Time','VisMetar']]
metar_df['Time'] = metar_df['Time'].str.slice(stop=-3)
sen_df['Time'] = sen_df['Time'].str.slice(stop=-3)
comp_df = pd.merge(comp_df,metar_df, on=['Date','Time'])



/tmp/ipykernel_7720/941557112.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,10,11,12,17,20,22,23,25,26,27,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  metar_df = pd.read_csv('Data/metar_speci_LZTT_2018_2021_decoded.csv')


,Date,Time,VisObs,MinVisObs,MinVisDirObs,MaxDist,VisSen,Difference,VisMetar
0,2019-11-01,05:30,10000,1300,SE,10000.0,10000,0.0,9999.0
1,2019-11-01,06:00,10000,1900,NE,10000.0,10000,0.0,9999.0
2,2019-11-01,06:30,7900,2900,E,10000.0,10000,0.0,9999.0
3,2019-11-01,07:00,5900,1900,S,10000.0,10000,0.0,9999.0
4,2019-11-01,07:30,10000,6800,SE,10000.0,10000,0.0,9999.0
...,...,...,...,...,...,...,...,...,...
215,2020-02-29,14:30,10000,6800,SE,10000.0,10000,0.0,10000.0
216,2020-02-29,15:00,10000,6800,SE,10000.0,10000,0.0,10000.0
217,2020-02-29,15:30,10000,6800,SE,8900.0,10000,1100.0,10000.0
218,2020-02-29,16:00,10000,6800,SE,10000.0,10000,0.0,10000.0


In [80]:
comp_df['Diff(METAR,YOLO)'] = (comp_df['VisMetar'] - comp_df['MaxDist']).abs()
comp_df.dropna(inplace=True)
comp_df['Diff(METAR,YOLO)'] = comp_df['Diff(METAR,YOLO)'].astype(int)
comp_df[comp_df['Diff(METAR,YOLO)'] > 1000]

,Date,Time,VisObs,MinVisObs,MinVisDirObs,MaxDist,VisSen,Difference,VisMetar,"Diff(METAR,YOLO)"
20,2019-11-02,05:30,8000,4200,E,8900.0,10000,1100.0,10000.0,1100
21,2019-11-02,06:00,8900,5900,S,8900.0,10000,1100.0,10000.0,1100
22,2019-11-02,06:30,8900,4900,SW,8900.0,10000,1100.0,10000.0,1100
23,2019-11-02,07:00,8900,5900,S,8900.0,10000,1100.0,10000.0,1100
24,2019-11-02,07:30,8000,5900,S,8900.0,10000,1100.0,10000.0,1100
25,2019-11-02,08:00,7800,4900,SW,8900.0,10000,1100.0,10000.0,1100
26,2019-11-02,08:30,8900,5900,S,8900.0,10000,1100.0,10000.0,1100
27,2019-11-02,09:00,8000,5900,S,8900.0,10000,1100.0,10000.0,1100
28,2019-11-02,09:30,8000,4900,S,8900.0,10000,1100.0,10000.0,1100
31,2019-11-02,11:00,8000,5900,S,8900.0,10000,1100.0,10000.0,1100
